In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [3]:

#preprocessing

#download csv dengan proses AUG
!gdown "1-1WvTPq90GbZxEFTKO65Oto5elsJRdHG"
#!gdown "1b60GkTBMN-NuqImLIOw0dph5R2EMgzQn"

Downloading...
From: https://drive.google.com/uc?id=1-1WvTPq90GbZxEFTKO65Oto5elsJRdHG
To: /content/full_AUG.csv
100% 159M/159M [00:00<00:00, 287MB/s]


In [4]:
import csv
from tqdm import tqdm
import ast
import re
import numpy as np
from sklearn.model_selection import train_test_split

# Path data CSV
path1 = "/content/full_AUG.csv"

# Persentase untuk train_set
persen_train_set = 0.98

kalimat_ = []
score_ = []

# Membaca data dari berbagai file CSV dan langsung membaginya ke dalam train_set dan val_set
with open(path1, 'r') as file_csv:
    csv_reader = csv.reader(file_csv)
    for baris in tqdm(csv_reader):
        kalimat_baru = re.sub(r'[-.,:;!?\n@#]|//t', ' ', baris[0])  # Preprocessing kalimat
        # String ke list
        skor_baru = np.array(ast.literal_eval(baris[1]))
        kalimat_.append(kalimat_baru)
        score_.append(skor_baru)

# Memutuskan apakah data akan dimasukkan ke dalam train_set atau val_set menggunakan sklearn train_test_split
train_kalimat, val_kalimat, train_score, val_score = train_test_split(kalimat_, score_, train_size=persen_train_set, random_state=42)

# Menampilkan data training
print("\nData training:", len(train_kalimat))
for i in range(min(3, len(train_kalimat))):
    print(train_kalimat[i])
print("\n{'neg':, 'neu':, 'pos':,}")
print(train_score[:3])

# Menampilkan data val
print("\nData val:", len(val_kalimat))
for i in range(min(3, len(val_kalimat))):
    print(val_kalimat[i])
print("\n{'neg':, 'neu':, 'pos':,}")
print(val_score[:3])


2432229it [00:43, 56367.91it/s]



Data training: 2383584
shan1392 I miss Gravity
helpspb Duh I'm watching mishacollins shirtless Highly recommended
emilioacosta really i do agree but i dunno how go there got apply visa everything but i DEFINITELY want go

{'neg':, 'neu':, 'pos':,}
[array([1, 0, 0]), array([0, 0, 1]), array([0, 0, 1])]

Data val: 48645
allyrockstar ok I get it I'm not badass
Go control maythethoughts MentalHealthAwarenessWeek ThoughtOfTheDay https /
worst cramp ever my hip

{'neg':, 'neu':, 'pos':,}
[array([0, 0, 1]), array([0, 1, 0]), array([1, 0, 0])]


In [5]:
print(len(train_kalimat),len(train_score))
print(len(val_kalimat),len(val_score))

2383584 2383584
48645 48645


In [6]:
#Tokenization and Padding
#Tokenization
max_words = 100000  # Set maximum number of words to consider
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_kalimat)

# Convert text sequences to integer sequences
train_sequences = tokenizer.texts_to_sequences(train_kalimat)
val_sequences = tokenizer.texts_to_sequences(val_kalimat)


# Padding sequences to ensure uniform length
max_length = 200  # Set maximum length of sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding='post', truncating='post')

In [7]:
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer

train_padded_resampled, train_score_resampled = np.array(train_padded), np.array(train_score)
val_padded_resampled, val_score_resampled = np.array(val_padded), np.array(val_score)


# Cek jumlah tiap kelas
netral = 0
positif = 0
negatif = 0

kelas = ["neg", "neu", "pos"]

for h in train_score_resampled:
    tip = np.argmax(h)
    if kelas[tip] == "neg":
        negatif += 1
    if kelas[tip] == "pos":
        positif += 1
    if kelas[tip] == "neu":
        netral +=1

print("Jumlah semuanya : ", len(train_score_resampled))
print("Netral : ", (netral))
print("Positif : ", (positif))
print("Negatif : ", (negatif))


Jumlah semuanya :  2383584
Netral :  794320
Positif :  794453
Negatif :  794811


In [ ]:
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime

# Model architecture
embedding_dim = 16

# Define the log directory
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

# Create TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


model = keras.Sequential([
    keras.layers.Embedding(max_words, embedding_dim, input_length=max_length),

    keras.layers.Bidirectional(keras.layers.LSTM(max_length)) ,
    keras.layers.Flatten(),

    keras.layers.Dense(256,activation = "relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(256,activation = "relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(256,activation = "relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(3, activation='softmax')  # Assuming output classes are 3 (neg, neu, pos)
])

optimizer  = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])



# Train the model,
num_epochs = 10
batch = 2200
model.fit(train_padded_resampled, train_score_resampled, epochs=num_epochs, validation_data=(val_padded_resampled, val_score_resampled), batch_size = batch, callbacks=[tensorboard_callback])

Epoch 1/10
1084/1084 [==============================] - 555s 505ms/step - loss: 0.4330 - accuracy: 0.8044 - val_loss: 0.3629 - val_accuracy: 0.8432
Epoch 2/10
1084/1084 [==============================] - 554s 511ms/step - loss: 0.3622 - accuracy: 0.8517 - val_loss: 0.3512 - val_accuracy: 0.8488
Epoch 3/10
1084/1084 [==============================] - 555s 512ms/step - loss: 0.3168 - accuracy: 0.8645 - val_loss: 0.3458 - val_accuracy: 0.8524
Epoch 4/10
1084/1084 [==============================] - 554s 511ms/step - loss: 0.2999 - accuracy: 0.8720 - val_loss: 0.3468 - val_accuracy: 0.8532
Epoch 5/10
1084/1084 [==============================] - 554s 511ms/step - loss: 0.2861 - accuracy: 0.8783 - val_loss: 0.3495 - val_accuracy: 0.8538
Epoch 6/10
1084/1084 [==============================] - 554s 511ms/step - loss: 0.2737 - accuracy: 0.8838 - val_loss: 0.3655 - val_accuracy: 0.8534
Epoch 7/10
  58/1084 [>.............................] - ETA: 8:38 - loss: 0.2498 - accuracy: 0.8964

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_padded_resampled, val_score_resampled, batch_size = 128)
print(f'Validation accuracy: {accuracy * 100:.2f}%')

In [15]:
#test_set


test_set = ["I don't know how to tell them that I have feelings for my best friend. It's tearing me apart inside.",
            "Love is complicated, and I feel lost in this sea of emotions. Does he/she feel the same way?",
            "The pressure at work is overwhelming. I've been putting in extra hours, but it never seems to be enough.",
            "I'm stuck in a job that I hate. I feel like I've lost all passion for what I do.",
            "My friends have been distant lately, and I don't know what I did wrong. I miss the times when we were so close.",
            "My relationship with my parents is strained, and I wish I could make them understand my point of view.",
            "I can't believe I found someone who understands me completely. Every moment with them feels like a fairytale.",
            "Spending the day doing what I love, whether it's reading, painting, or hiking, fills my heart with happiness.",
            "The car is blue.",
            "My mental health is deteriorating day by day. Dealing with depression feels like being trapped in a never-ending cycle of darkness. I don't know how to escape it."
            ]

kelas = ["neg", "neu", "pos"]

while True :

#for h in test_set :
  h = input("Masukan kalimat (English) : ")
  toke = tokenizer.texts_to_sequences([h])
  pad = pad_sequences(toke, maxlen=max_length, padding='post', truncating='post')

  output = model.predict(pad)
  print(output)


  print("input : ", h , "\n - prediksi => ", kelas[np.argmax(output)])
  print()

Masukan kalimat (English) : the car is yellow
1/1 [==============================] - 0s 23ms/step
[[0.5108112  0.03248708 0.4567017 ]]
input :  the car is yellow 
 - prediksi =>  neg



KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
import pickle
import shutil

# Simpan model
nama = "model_aug_dropout"
num_epochs = 10  # Replace with your actual value

model.save("/content/drive/MyDrive/DATASET_CAPSTONE/model/" + nama + "/" + nama + "_epoch_" + str(num_epochs) + ".h5")

# Simpan tokenizer
with open("/content/drive/MyDrive/DATASET_CAPSTONE/model/" + nama + "/" + "tokenizer_" + nama + "_epoch_" + str(num_epochs) + ".pk", 'wb') as file:
    pickle.dump(tokenizer, file)

# Simpan loss
logs_dir = "/content/logs/"
destination_dir = "/content/drive/MyDrive/DATASET_CAPSTONE/model/" + nama + "[LOSS]_epoch_" + str(num_epochs)

shutil.copytree(logs_dir, destination_dir)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


'/content/drive/MyDrive/DATASET_CAPSTONE/model/model_aug_dropout[LOSS]_epoch_10'